# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [5]:
%pip install sentence-transformers
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [6]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [7]:
K = 25

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df_news = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MNA/FZ4025.10 Fintech/Tareas/S3/Project 2/df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [10]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [11]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.


In [12]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'],
        'TICKER': row['TICKER'],
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [13]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [14]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [15]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [16]:
from google.colab import userdata
OpenAI_api_key = userdata.get('OpenAI_API_Key')

In [17]:
# Defining and using a language model agent for fed policy Q & A
# Define and test and openAI LLM Client
from openai import OpenAI
client = OpenAI(api_key=OpenAI_api_key)

response = client.responses.create(
    model="gpt-4o-mini",
    input="Write a one-sentence bedtime story about a unicorn."
)
print(response.output_text)

As the moonlight twinkled like diamonds, Luna the unicorn gracefully danced through the enchanted forest, spreading magical dreams to all the sleeping creatures around her.


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [18]:
%pip install yfinance

In [19]:
unique_tickers = df_news['TICKER'].unique()
display(unique_tickers)

array(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL',
       'A', 'APD', 'ABNB', 'AKAM', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL',
       'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG',
       'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON',
       'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG',
       'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON',
       'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BBY', 'TECH', 'BIIB', 'BLK',
       'BX', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO',
       'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF',
       'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR',
       'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CHD',
       'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS',
       'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ',
       'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST',

In [20]:
import yfinance as yf
import time

company_metadata = []

for ticker in unique_tickers:
    try:
        ticker_yf = yf.Ticker(ticker)
        info = ticker_yf.info

        company_name = info.get('longName', 'N/A')
        sector = info.get('sector', 'N/A')
        industry = info.get('industry', 'N/A')

        company_metadata.append({
            'TICKER': ticker,
            'COMPANY_NAME': company_name,
            'SECTOR': sector,
            'INDUSTRY': industry
        })
        time.sleep(1)  # Add a small delay to avoid hitting API rate limits

    except Exception as e:
        print(f"Could not retrieve data for {ticker}: {e}")
        company_metadata.append({
            'TICKER': ticker,
            'COMPANY_NAME': 'N/A',
            'SECTOR': 'N/A',
            'INDUSTRY': 'N/A'
        })


In [21]:
df_meta = pd.DataFrame(company_metadata)
display(df_meta)

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services
...,...,...,...,...
485,XEL,Xcel Energy Inc.,Utilities,Utilities - Regulated Electric
486,XYL,Xylem Inc.,Industrials,Specialty Industrial Machinery
487,YUM,"Yum! Brands, Inc.",Consumer Cyclical,Restaurants
488,ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,Medical Devices


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [22]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [23]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [24]:
# Define the RAG function
def rag(query, k=K):
    """
    Retrieves relevant documents from the FAISS store and formats them for the LLM prompt.
    """
    retrieved_docs = faiss_store.search(query, k=k)

    # Format the retrieved documents for the LLM prompt
    context = "\n\n".join([doc[0] for doc in retrieved_docs])

    return context, retrieved_docs

In [25]:
# Generate the answers with OpenAI client
def generate_answer(query, context):
    """
    Generates an answer to the query based on the provided context using the OpenAI model.
    """
    prompt = f"""
    Based on the following news articles, answer the question:

    {context}

    Question: {query}

    Provide a concise answer based only on the information provided in the articles.
    """
    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt
    )
    return response.output_text

In [26]:
# Create the RAG pipeline that takes the previous steps into one function that takes a question and returns the answer and the source docs
def run_rag_pipeline(query, k=K):
    """
    Runs the RAG pipeline to retrieve relevant documents and generate an answer.
    """
    context, retrieved_docs = rag(query, k=k)
    answer = generate_answer(query, context)
    return answer, retrieved_docs

In [27]:
question_lists = {
    "Topic Questions": questions_topic,
    "Company Questions": questions_company,
    "Industry Questions": questions_industry
}

for list_name, questions in question_lists.items():
    print(f"--- {list_name} ---")
    for question in questions:
        print(f"\nQuestion: {question}")
        answer, retrieved_docs = run_rag_pipeline(question)
        print_markdown(f"**Answer:**\n{answer}")
        print("\nRetrieved Documents:")
        for i, (doc_text, doc_meta, sim) in enumerate(retrieved_docs):
            print(f"{i+1}. Similarity: {sim:.4f}")
            print(f"   Title/Summary: {doc_text.split(':', 1)[0] if ':' in doc_text else doc_text}")
            print(f"   Metadata: Ticker: {doc_meta['TICKER']}, Date: {doc_meta['PUBLICATION_DATE']}, Provider: {doc_meta['PROVIDER']}")
            print(f"   Summary: {doc_text.split(':', 1)[1].strip() if ':' in doc_text else 'N/A'}")
        print("-" * 50)

--- Topic Questions ---

Question: What are the major concerns expressed in financial news about inflation?


**Answer:**
The major concerns expressed in the financial news about inflation include:

1. **Persistent Inflation Risks**: The Federal Reserve's recent discussions indicate growing anxiety over sustained inflation and its potential impact on the economy.

2. **Food Inflation and Rate Cuts**: Rising food inflation is dampening hopes for interest rate cuts, emphasizing how current tariffs are affecting grocery prices.

3. **Economic Slowdown**: There is apprehension that ongoing inflation could lead to an economic slowdown, which is complicating the earnings outlook and causing analysts to lower projections.

4. **Fiscal Health Concerns**: The U.S. national debt and increasing interest payments are leading to fears about fiscal sustainability, prompting investors to seek stability in gold and other hard assets.

5. **Impact on Consumer Stocks**: Consumer discretionary stocks are struggling due to inflation-related uncertainty, with recent declines in their performance highlighting sensitivity to economic cycles.


Retrieved Documents:
1. Similarity: 0.5771
   Title/Summary: Bitcoin price slips as Fed minutes flag US inflation risks 
   Metadata: Ticker: BLK, Date: 2025-05-29, Provider: Yahoo Finance UK
   Summary: The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.
2. Similarity: 0.4920
   Title/Summary: The Weekend
   Metadata: Ticker: TSLA, Date: 2025-05-31, Provider: Yahoo Finance UK
   Summary: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
3. Similarity: 0.4920
   Title/Summary: The Weekend
   Metadata: Ticker: NVDA, Date: 2025-05-31, Provider: Yahoo Finance UK
   Summary: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
4. Similarity: 0.4920
   Title/Summary: The Weekend
   Metadata: Ticker: LULU, Dat

**Answer:**
Investor sentiment in recent financial headlines is described as cautious and skeptical, particularly towards stocks that face headwinds or have bearish forecasts from Wall Street. Many articles highlight that analysts often issue overly optimistic or favorable ratings due to institutional pressures, raising concerns about the reliability of these forecasts. Stocks receiving bearish ratings are noted as significant, as such views are uncommon in the industry, further emphasizing caution among investors.


Retrieved Documents:
1. Similarity: 0.6115
   Title/Summary: 3 of Wall Street’s Favorite Stocks Facing Headwinds 
   Metadata: Ticker: KMX, Date: 2025-05-26, Provider: StockStory
   Summary: Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts.
2. Similarity: 0.5978
   Title/Summary: 3 Hyped Up  Stocks Facing Headwinds 
   Metadata: Ticker: MCHP, Date: 2025-05-20, Provider: StockStory
   Summary: Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive news flow, or even a loyal Reddit fanbase.
3. Similarity: 0.5894
   Title/Summary: 1 of Wall Street’s Favorite Stock with Impressive Fundamentals and 2 to Think Twice About 
   Metadata: Ticker: MPWR, Date

...


Retrieved Documents:
1. Similarity: 0.6429
   Title/Summary: Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends 
   Metadata: Ticker: ON, Date: 2025-05-13, Provider: Zacks
   Summary: Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock.
2. Similarity: 0.5822
   Title/Summary: ON Semiconductor Corporation (ON) is Attracting Investor Attention
   Metadata: Ticker: ON, Date: 2025-05-21, Provider: Zacks
   Summary: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store.
3. Similarity: 0.5649
   Title/Summary: Some May Be Optimistic About ON Semiconductor's (NASDAQ
   Metadata: Ticker: ON, Date: 2025-05-12, Provider: Simply Wall St.
   Summary: ON) Earnings : Soft earnings didn't appear to concern ON Semiconductor Corpo

**Answer:**
The retail industry is experiencing volatility with stocks tumbling, showing a decline of 13.7% over the past six months, which is worse than the S&P 500’s 5.5% loss. Retailers are adapting their business models in response to changing consumer shopping behaviors due to technology and have shifted supply chains amid ongoing trade war challenges. There is also concern over potential inventory overflow due to a tariff pause, which could result in deeper discounts and margin pressure. Overall, the demand for retail products appears to be fading.


Retrieved Documents:
1. Similarity: 0.5898
   Title/Summary: 3 Consumer Stocks That Concern Us 
   Metadata: Ticker: KMX, Date: 2025-05-12, Provider: StockStory
   Summary: Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.
2. Similarity: 0.5056
   Title/Summary: Retailers, Ducking Trade-War Curveballs, Stick to Their Plans 
   Metadata: Ticker: BBY, Date: 2025-05-29, Provider: The Wall Street Journal
   Summary: As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves.
3. Similarity: 0.4997
   Title/Summary: 3 Consumer Stocks Skating on Thin Ice 
   Metadata: Ticker: HLT, Date: 2025-05-22,

**Answer:**
Recent news articles highlight several risks and challenges facing the energy industry:

1. **Legislative Threats**: A bill in Congress could repeal crucial subsidies for the renewable energy sector, making projects economically unfeasible and leading to a crash in renewable energy stocks.

2. **Market Volatility**: The oilfield service sector is struggling with sliding oil prices, rising tariffs, and shrinking drilling budgets, raising concerns about profitability.

3. **Tariff Impacts**: Ongoing trade conflicts, particularly related to tariffs, create uncertainty in the industrial and energy sectors, complicating competitive positioning.

4. **Economic Downturn**: Industrial stocks are forecasted to suffer due to high capital requirements and market expectations of a prolonged economic downturn, with significant declines reported in stock performance.

5. **Litigation Risks**: Companies like Edison International face legal challenges due to misleading statements regarding wildfire risks, which can further impact their operations and investor confidence.

6. **Undervalued Stocks**: A substantial portion of mid- and small-cap oil and gas stocks are trading below book value, indicating potential undervaluation and fears of long-term profitability issues.

7. **Environmental and Regulatory Hurdles**: Projects like Dominion Energy’s offshore wind initiative face uncertainties due to political opposition, impacting potential job creation and energy needs.

Overall, the energy sector is navigating a complex landscape of economic, regulatory, and competitive challenges.


Retrieved Documents:
1. Similarity: 0.5362
   Title/Summary: Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry 
   Metadata: Ticker: NEE, Date: 2025-05-23, Provider: Motley Fool
   Summary: Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.
2. Similarity: 0.5362
   Title/Summary: Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry 
   Metadata: Ticker: ENPH, Date: 2025-05-23, Provider: Motley Fool
   Summary: Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped 

## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance**
2. **Completeness**  
3. **Bias or Noise**
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

In most of the cases the top 5 news snippets support the generated answer, but as we move on to the rest of the news with less similarity score we start to find some unrelated information about the question and answers provided.
The key facts or themes in the answers are generally grounded in the context, mentioning specific terms included in the questions.


There is some variation on the level of support depending on the question and the specific snippets retrieved. Some snippets are more relevant and informative than others, leading to a better answer while other points might be based on weaker or less explicit information from the snippets.

#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

The answers generally address the questions, but there may be some aspects that are left out, this could be because of the lack of completeness found on the news or that the contexts are weak for specific questions.

By adding more context to the news (metadata), we can get better and more thourough responses.

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

Yes, there can be irrelevant or misleading snippets retrieved. This can happen if we include all the different news providers, without applying a specific filter (some news providers could be less serious than others).  
The semantic search can find relevant documents based on the matching keywords but they can have different contexts that can affect the similarity score.


Irrelevant snippets can negatively affect the quality of the output in the following ways:
*   **Affecting relevant information:** They can push relevant snippets out of the top K results.
*   **Introducing noise:** The LLM might include information from these snippets that is not pertinent to the question, making the answer less complete or relevant.
*   **Potential inaccuracies:** In some cases, misleading snippets could create an answer that is partially inaccurate, especially if conflicting information is present.

#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

The news snippets can sometimes show inconsistent information or different viewpoints, for example when discussing the market sentiment.

The LLM helped summarized the news but it didn't necessarily handle potential contradictions. The contradictions can be included on the answers.

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

Some suggestions to improve the generation process are:

*   **Improve filtering:** Include a metadata filter to get more relevant information (by sector, date, etc).
*   **Try different `k`:** With a smaller `k` we could reduce noise but also miss relevant information. With a larger `k` we could include noise and make the prompt take longer. We can try different values to see what makes the answers more grounded and relevant.
*   **Try different prompts :** Improve the prompt to the LLM to better handle potential inconsistencies and focus on synthesizing information. For example, asking the LLM to identify different viewpoints if present and instruct the LLM to prioritize information from more reputable sources.
*   **Re-ranking Retrieved Documents:** After initial retrieval, re-rank the documents based on additional criteria, such as publication date (prioritizing newer news) or source reliability, before passing them to the LLM.
*   **Group news by "title/summary":** Some of the news snippets were repeated because they would include different tickers. This could give more importance to them and leave more relevant news out of the context.

## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [28]:
# Define the RAG function with metadata enrichment
def rag_metadata(query, df_meta, k=K):
    """
    Retrieves relevant documents from the FAISS store and formats them for the LLM prompt,
    including financial metadata.
    """
    retrieved_docs = faiss_store.search(query, k=k)

    enriched_docs = []
    context_parts = []

    for doc_text, doc_meta, sim in retrieved_docs:
        ticker = doc_meta.get('TICKER')
        company_name = 'N/A'
        sector = 'N/A'
        industry = 'N/A'

        if ticker and ticker in df_meta['TICKER'].values:
            meta_row = df_meta[df_meta['TICKER'] == ticker].iloc[0]
            company_name = meta_row.get('COMPANY_NAME', 'N/A')
            sector = meta_row.get('SECTOR', 'N/A')
            industry = meta_row.get('INDUSTRY', 'N/A')

        # Create an enriched metadata dictionary
        enriched_meta = {
            'TICKER': ticker,
            'COMPANY_NAME': company_name,
            'SECTOR': sector,
            'INDUSTRY': industry,
            'PUBLICATION_DATE': doc_meta.get('PUBLICATION_DATE'),
            'PROVIDER': doc_meta.get('PROVIDER')
        }

        enriched_docs.append((doc_text, enriched_meta, sim))

        # Format context for the LLM, including metadata
        context_parts.append(
            f"Title/Summary: {doc_text}\n"
            f"Ticker: {enriched_meta['TICKER']}, Company: {enriched_meta['COMPANY_NAME']}, "
            f"Sector: {enriched_meta['SECTOR']}, Industry: {enriched_meta['INDUSTRY']}, "
            f"Date: {enriched_meta['PUBLICATION_DATE']}, Provider: {enriched_meta['PROVIDER']}"
        )

    context = "\n\n---\n\n".join(context_parts)

    return context, enriched_docs

# Update the run_rag_pipeline function to pass df_meta to rag
def run_rag_pipeline_with_metadata(query, df_meta, k=K):
    """
    Runs the RAG pipeline with metadata enrichment to retrieve relevant documents
    and generate an answer.
    """
    context, retrieved_docs = rag_metadata(query, df_meta, k=k)
    answer = generate_answer(query, context) # generate_answer uses the enriched context
    return answer, retrieved_docs

In [29]:
question_lists = {
    "Topic Questions (with Metadata)": questions_topic,
    "Company Questions (with Metadata)": questions_company,
    "Industry Questions (with Metadata)": questions_industry
}

for list_name, questions in question_lists.items():
    print(f"--- {list_name} ---")
    for question in questions:
        print(f"\nQuestion: {question}")
        # Use the updated RAG pipeline function
        answer, retrieved_docs = run_rag_pipeline_with_metadata(question, df_meta)
        print_markdown(f"**Answer (with Metadata):**\n{answer}")
        print("\nRetrieved Documents (with Metadata):")
        for i, (doc_text, doc_meta, sim) in enumerate(retrieved_docs):
            print(f"{i+1}. Similarity: {sim:.4f}")
            print(f"   Metadata: Ticker: {doc_meta['TICKER']}, Company: {doc_meta['COMPANY_NAME']}, Sector: {doc_meta['SECTOR']}, Industry: {doc_meta['INDUSTRY']}, Date: {doc_meta['PUBLICATION_DATE']}, Provider: {doc_meta['PROVIDER']}")
            # Print the original Title/Summary to keep it clean, as the full text is in the context sent to LLM
            print(f"   Original Text (Title/Summary): {doc_text}")
        print("-" * 50)

--- Topic Questions (with Metadata) ---

Question: What are the major concerns expressed in financial news about inflation?


**Answer (with Metadata):**
The major concerns expressed in the financial news about inflation include:

1. Persistent US inflation risks, contributing to potential economic slowdown (as highlighted by the Federal Reserve's minutes).
2. Food inflation dampening hopes for a rate cut, as ongoing tariff issues continue to affect prices.
3. General uncertainty about the macroeconomic outlook, leading to a negative impact on earnings expectations and stock performance.
4. Rising geopolitical tensions and skepticism about fiscal discipline, prompting investors to seek stability in hard assets amidst inflation concerns.
5. The overall economic cycle's effect on consumer discretionary stocks, as inflation pressures lead to reduced consumer spending.

These factors indicate an interconnected landscape of inflation affecting both economic forecasts and market behaviors.


Retrieved Documents (with Metadata):
1. Similarity: 0.5771
   Metadata: Ticker: BLK, Company: BlackRock, Inc., Sector: Financial Services, Industry: Asset Management, Date: 2025-05-29, Provider: Yahoo Finance UK
   Original Text (Title/Summary): Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown.
2. Similarity: 0.4920
   Metadata: Ticker: TSLA, Company: Tesla, Inc., Sector: Consumer Cyclical, Industry: Auto Manufacturers, Date: 2025-05-31, Provider: Yahoo Finance UK
   Original Text (Title/Summary): The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead
3. Similarity: 0.4920
   Metadata: Ticker: NVDA, Company: NVIDIA Corporation, Sector: Technology, Industry: Semiconductors, Date: 2025-05-31, Provider: Yahoo Finance UK
   Original Te

**Answer (with Metadata):**
Investor sentiment in recent financial headlines is largely mixed, characterized by a willingness to be optimistic about stocks favored by Wall Street, reflected in ambitious price targets and a generally bullish outlook. However, there is also caution, as many articles highlight headwinds, bearish forecasts, and the pressure on analysts to maintain positive ratings, suggesting an underlying skepticism regarding the sustainability of these optimistic views. Overall, while some stocks are experiencing momentum and positive catalysts, others are facing significant challenges and bear sentiments.


Retrieved Documents (with Metadata):
1. Similarity: 0.6115
   Metadata: Ticker: KMX, Company: CarMax, Inc., Sector: Consumer Cyclical, Industry: Auto & Truck Dealerships, Date: 2025-05-26, Provider: StockStory
   Original Text (Title/Summary): 3 of Wall Street’s Favorite Stocks Facing Headwinds : Wall Street has set ambitious price targets for the stocks in this article. While this suggests attractive upside potential, it’s important to remain skeptical because analysts face institutional pressures that can sometimes lead to overly optimistic forecasts.
2. Similarity: 0.5978
   Metadata: Ticker: MCHP, Company: Microchip Technology Incorporated, Sector: Technology, Industry: Semiconductors, Date: 2025-05-20, Provider: StockStory
   Original Text (Title/Summary): 3 Hyped Up  Stocks Facing Headwinds : Great things are happening to the stocks in this article. They’re all outperforming the market over the last month because of positive catalysts such as a new product line, constructive new

...


Retrieved Documents (with Metadata):
1. Similarity: 0.6429
   Metadata: Ticker: ON, Company: ON Semiconductor Corporation, Sector: Technology, Industry: Semiconductors, Date: 2025-05-13, Provider: Zacks
   Original Text (Title/Summary): Investing in ON Semiconductor Corp. (ON)? Don't Miss Assessing Its International Revenue Trends : Explore ON Semiconductor Corp.'s (ON) international revenue trends and how these numbers impact Wall Street's forecasts and what's ahead for the stock.
2. Similarity: 0.5822
   Metadata: Ticker: ON, Company: ON Semiconductor Corporation, Sector: Technology, Industry: Semiconductors, Date: 2025-05-21, Provider: Zacks
   Original Text (Title/Summary): ON Semiconductor Corporation (ON) is Attracting Investor Attention: Here is What You Should Know : Recently, Zacks.com users have been paying close attention to ON Semiconductor Corp. (ON). This makes it worthwhile to examine what the stock has in store.
3. Similarity: 0.5649
   Metadata: Ticker: ON, Company: O

**Answer (with Metadata):**
The retail industry is experiencing volatility, with consumer demand trends working against many businesses. Over the past six months, retail stocks have tumbled significantly, with declines ranging from 12.3% to 13.9%, worse than the S&P 500's losses. Retailers are adapting their supply chains and pricing strategies amid challenges like inventory overflow due to tariff pauses and economic cycles impacting discretionary spending. High inventory and labor costs further complicate operations, leaving little margin for error as demand wanes.


Retrieved Documents (with Metadata):
1. Similarity: 0.5898
   Metadata: Ticker: KMX, Company: CarMax, Inc., Sector: Consumer Cyclical, Industry: Auto & Truck Dealerships, Date: 2025-05-12, Provider: StockStory
   Original Text (Title/Summary): 3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred some uncertainty lately as retail stocks have tumbled by 13.7% over the past six months. This performance was worse than the S&P 500’s 5.5% loss.
2. Similarity: 0.5056
   Metadata: Ticker: BBY, Company: Best Buy Co., Inc., Sector: Consumer Cyclical, Industry: Specialty Retail, Date: 2025-05-29, Provider: The Wall Street Journal
   Original Text (Title/Summary): Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shift

**Answer (with Metadata):**
Recent news about the energy industry highlights several risks and challenges, including:

1. **Legislative Changes**: The U.S. Congress is advancing a bill that could repeal key subsidies for the renewable energy sector, threatening the economic viability of various renewable projects (NextEra Energy, Enphase Energy).

2. **Price and Demand Pressures**: Oilfield service companies face difficulties due to sliding oil prices, rising tariffs, and shrinking drilling budgets, potentially impacting industry profitability (Halliburton, Baker Hughes).

3. **Competition and Tariffs**: The trade conflict and tariff landscape remain uncertain, affecting stock valuations and competitive positioning for companies in the energy sector (Freeport-McMoRan, Schlumberger).

4. **Legal Liabilities**: Companies like Xcel Energy and Edison International are facing lawsuits related to wildfire risks, which could harm their reputations and financial stability.

5. **Economic Downturn Concerns**: The overall industrial sector, of which energy is a part, is predicting a downturn, as evidenced by significant stock pullbacks over recent months (Otis Worldwide, Dover Corporation).

6. **Valuation Concerns**: Many mid- and small-cap oil and gas stocks are trading below their book values, indicating potential financial instability and investor apprehension (Diamondback Energy).

These challenges underline a complex landscape for the energy industry amid changing regulations, market dynamics, and economic uncertainties.


Retrieved Documents (with Metadata):
1. Similarity: 0.5362
   Metadata: Ticker: NEE, Company: NextEra Energy, Inc., Sector: Utilities, Industry: Utilities - Regulated Electric, Date: 2025-05-23, Provider: Motley Fool
   Original Text (Title/Summary): Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may become uneconomical.  The news was about as bad as it could get for renewable energy stocks this week as the U.S. House of Representatives early Thursday passed a bill that will repeal some of the most important subsidies for the industry if it becomes law.
2. Similarity: 0.5362
   Metadata: Ticker: ENPH, Company: Enphase Energy, Inc., Sector: Technology, Industry: Solar, Date: 2025-05-23, Provider: Motley Fool
   Original Text (Title/Summary): Renewable Energ

## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



In [30]:
print_markdown("""
| Criteria         | WITHOUT METADATA                                                                                                | WITH METADATA                                                                                                                               |
|------------------|-----------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------|
| Clarity          | Answers are generally clear, but sometimes lack specific context.                                  | Answers are clear and often provide more specific context by referencing company names, sectors, and industries from the metadata. |
| Detail & Depth   | Provides a good overview based on the text, but can miss information related to specific companies or industries.     | Offers more detailed answers by including company and industry information, allowing for a better understanding.          |
| Use of Context   | Uses the text of the articles effectively to form the answer.                                                     | Uses both the text and the provided metadata (Ticker, Company, Sector, Industry) to form the answer, providing more complete context.          |
| Accuracy & Grounding| Grounded in the retrieved text, but accuracy can be limited by the lack of company/sector identification in the context. | More accurately connects information to companies and sectors due to the explicit metadata provided in the context.         |
| Relevance        | Generally relevant to the query based on semantic similarity of the text.                                         | Relevance is enhanced by the metadata, ensuring the answer is related to specific companies or industries mentioned in the retrieved context.   |
| Narrative Flow   | The answer can be read as a summary of the retrieved snippets.                                                                   | The answer reads as a structured summary, organizing information by company or industry, improving the narrative flow and readability.     |
""")


| Criteria         | WITHOUT METADATA                                                                                                | WITH METADATA                                                                                                                               |
|------------------|-----------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------|
| Clarity          | Answers are generally clear, but sometimes lack specific context.                                  | Answers are clear and often provide more specific context by referencing company names, sectors, and industries from the metadata. |
| Detail & Depth   | Provides a good overview based on the text, but can miss information related to specific companies or industries.     | Offers more detailed answers by including company and industry information, allowing for a better understanding.          |
| Use of Context   | Uses the text of the articles effectively to form the answer.                                                     | Uses both the text and the provided metadata (Ticker, Company, Sector, Industry) to form the answer, providing more complete context.          |
| Accuracy & Grounding| Grounded in the retrieved text, but accuracy can be limited by the lack of company/sector identification in the context. | More accurately connects information to companies and sectors due to the explicit metadata provided in the context.         |
| Relevance        | Generally relevant to the query based on semantic similarity of the text.                                         | Relevance is enhanced by the metadata, ensuring the answer is related to specific companies or industries mentioned in the retrieved context.   |
| Narrative Flow   | The answer can be read as a summary of the retrieved snippets.                                                                   | The answer reads as a structured summary, organizing information by company or industry, improving the narrative flow and readability.     |
